In [1]:
import pymongo as pmn
from sshtunnel import SSHTunnelForwarder
import pprint
import datetime
import pandas as pd
from tqdm import tqdm_notebook
import numpy as np

In [2]:
MONGO_HOST = "remote.vdi.mipt.ru"
MONGO_DB = "VectorX_DWH"
MONGO_USER = "team2"
MONGO_PASS = "gloche2"
MONGO_PORT = 65372

In [3]:
server = SSHTunnelForwarder(
    ssh_address_or_host=(MONGO_HOST, MONGO_PORT),
    ssh_username="team2",
    ssh_password="gloche2",
    remote_bind_address=('127.0.0.1', 27017),
)

server.start()

client = pmn.MongoClient('127.0.0.1', server.local_bind_port) # server.local_bind_port is assigned local port
db = client[MONGO_DB]
pprint.pprint(db.collection_names())
server.stop()

['documents', 'topic_by_date']


In [143]:
server.stop()

In [144]:
server.start()
client = pmn.MongoClient('127.0.0.1', server.local_bind_port) # server.local_bind_port is assigned local port
db = client[MONGO_DB]

In [90]:
lvl1_names = set()
cursor = db.documents.find({}).limit(1000)
for i, row in tqdm_notebook(enumerate(cursor)):
    if len(row['topics']) > 0:
        lvl1_names.add(row['topics'][0]['lvl1']['id'])


HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

In [146]:
lvl1_names

{'lvl1_бизнес',
 'lvl1_культура',
 'lvl1_медиа',
 'lvl1_наука_и_техника',
 'lvl1_общество_и_путешествия',
 'lvl1_политика',
 'lvl1_происшествия',
 'lvl1_силовые_структуры',
 'lvl1_спорт',
 'lvl1_экономика_и_финансы'}

In [147]:
second_to_first = {key:value for (key, value) in zip(list(lvl1_names), [pd.DataFrame()] * len(lvl1_names))}


cursor = db.documents.find({}).limit(13000)
for i, row in tqdm_notebook(enumerate(cursor)):
    if len(row['topics']) > 0:
        lvl2_names.add(row['topics'][0]['lvl2']['topic_label'])


HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

KeyboardInterrupt: 

In [91]:
lvl1_names

{'lvl1_бизнес',
 'lvl1_культура',
 'lvl1_медиа',
 'lvl1_наука_и_техника',
 'lvl1_общество_и_путешествия',
 'lvl1_политика',
 'lvl1_происшествия',
 'lvl1_силовые_структуры',
 'lvl1_спорт',
 'lvl1_экономика_и_финансы'}

In [104]:
lvl1_words = set()
cursor = db.documents.find({}).limit(10000)
for i, row in tqdm_notebook(enumerate(cursor)):
    if len(row['topics']) > 0:
        dtc = row['topics'][0]['lvl1']['distribution']
        for gword in dtc:
            lvl1_words.add(gword['word'])

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

In [126]:
lvl1_words = sorted(list(lvl1_words))[160:-10]

In [83]:
lvl1_words = set()
cursor = db.documents.find({}).limit(10000)
for i, row in tqdm_notebook(enumerate(cursor)):
    pprint.pprint(row)
    break
    if len(row['topics']) > 0:
        lvl1_words.add(row['topics'][0]['lvl2']['topic_label'])


HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

{'_id': ObjectId('5a835af37ea3d4320cb4fc9a'),
 'auditory': {'versions': [{'ondate': datetime.datetime(2018, 2, 28, 17, 23, 29, 396000),
                            'segments': [{'affinity': 59.50078963900685,
                                          'id': '65',
                                          'views': 1887,
                                          'visitors': 1025},
                                         {'affinity': 178.947140102193,
                                          'id': '2241',
                                          'views': 90,
                                          'visitors': 79},
                                         {'affinity': 100.07247974856269,
                                          'id': '38',
                                          'views': 782,
                                          'visitors': 434},
                                         {'affinity': 0.0,
                                          'id': '0',
                     

In [99]:
lvl1_names

{'lvl1_бизнес',
 'lvl1_культура',
 'lvl1_медиа',
 'lvl1_наука_и_техника',
 'lvl1_общество_и_путешествия',
 'lvl1_политика',
 'lvl1_происшествия',
 'lvl1_силовые_структуры',
 'lvl1_спорт',
 'lvl1_экономика_и_финансы'}

In [161]:
second_to_first = {key:value for (key, value) in zip(list(lvl1_names), [pd.DataFrame()] * len(lvl1_names))}
start_time = datetime.datetime(2017, 8, 1, 0, 0, 0)
delta = datetime.timedelta(hours=2)

while start_time + delta < datetime.datetime(2017, 9, 1, 0, 0, 0):
    cursor = db.documents.find({"rss_pubDate" : {'$gt' : start_time, '$lt' : start_time + delta}}).limit(50)
    cur_dic = {key : dict() for key in list(lvl1_names)}
    for row in cursor:
        
        if len(row['topics']) > 0:
            l1 = row['topics'][0]['lvl1']['id']
            l2 = row['topics'][0]['lvl2']['topic_label']
            if l2 not in cur_dic[l1].keys():
                cur_dic[l1][l2] = 0
            cur_dic[l1][l2] += 1
    #print(cur_dic)
    for key in cur_dic:
        second_to_first[key] = second_to_first[key].append(cur_dic[key], ignore_index=True)
    start_time += delta

In [162]:
second_to_first[list(lvl1_names)[4]]

,кино,"телевидение, радио",знаменитости
0,1.0,NaN,NaN
1,1.0,1.0,NaN
2,NaN,1.0,1.0
3,NaN,1.0,NaN
4,NaN,NaN,1.0
5,2.0,1.0,1.0
6,3.0,1.0,NaN
7,NaN,2.0,NaN
8,NaN,NaN,NaN
9,1.0,NaN,NaN


In [179]:
buffer = second_to_first
for key in buffer.keys():
    buffer[key] = buffer[key].fillna(value=0)
    buffer[key] = buffer[key].apply(lambda x : x / np.sum(x), axis=1)
    buffer[key] = buffer[key].fillna(value=0)
    buffer[key].to_csv('./second_to_first_' + key + '_.csv', index_label='ID')

In [180]:
buffer['lvl1_силовые_структуры'].head()

,армии государств,полиция
0,0.000000,0.000000
1,0.000000,0.000000
2,0.000000,0.000000
3,0.257396,0.742604
4,0.000000,1.000000


In [129]:
lvl1 = pd.DataFrame()
start_time = datetime.datetime(2017, 8, 1, 0, 0, 0)
delta = datetime.timedelta(hours=2)

while start_time + delta < datetime.datetime(2017, 9, 1, 0, 0, 0):
    cursor = db.documents.find({"rss_pubDate" : {'$gt' : start_time, '$lt' : start_time + delta}}).limit(100)
    cur_dic = {key : value for (key, value) in zip(list(lvl1_words), np.zeros(len(lvl1_words)))}
    cur_dic['total'] = 0
    for row in cursor:
        
        if len(row['topics']) > 0:
            dtc = row['topics'][0]['lvl1']['distribution']
            for gword in dtc:
                if gword['word'] in cur_dic.keys():
                    cur_dic[gword['word']] += gword['prob']
                    cur_dic['total'] += 1
    lvl1 = lvl1.append(cur_dic, ignore_index=True)
    start_time += delta

In [142]:
lvl1.head()

,android,associated,august,az,bmw,by,cnn,co,createelement,ctrl,...,яркий,яркий_пример,яркий_событие,ярмарка,ярослав,ярославль,ярославский,ярославский_область,ясно,ясный
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.000141,0.000000,0.0,0.0,0.000000,0.0
1,0.0,0.0,0.0,0.0,0.0,0.000123,0.0,0.0,0.000000,0.000475,...,0.000242,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000105,0.0
2,0.0,0.0,0.0,0.0,0.0,0.000123,0.0,0.0,0.000179,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000179,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000105,0.0
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.000385,0.000000,0.000165,0.0,0.0,0.000105,0.0


2018-03-04 15:27:55,051| ERROR   | Could not establish connection from ('127.0.0.1', 39837) to remote side of the tunnel
2018-03-04 15:28:06,070| ERROR   | Could not establish connection from ('127.0.0.1', 39837) to remote side of the tunnel
2018-03-04 15:28:17,087| ERROR   | Could not establish connection from ('127.0.0.1', 39837) to remote side of the tunnel
2018-03-04 15:28:28,105| ERROR   | Could not establish connection from ('127.0.0.1', 39837) to remote side of the tunnel
2018-03-04 15:28:39,126| ERROR   | Could not establish connection from ('127.0.0.1', 39837) to remote side of the tunnel


In [131]:
lvl1.to_csv('august_2h_100_words.csv')

In [138]:
buffer = lvl1
buffer = buffer.drop(columns='total', axis=0)
buffer = buffer.apply(lambda x : x / np.sum(x.values) , axis=1)
buffer.to_csv('august_2h_100_words_norm.csv')

In [141]:
buffer.head()

,android,associated,august,az,bmw,by,cnn,co,createelement,ctrl,...,яркий,яркий_пример,яркий_событие,ярмарка,ярослав,ярославль,ярославский,ярославский_область,ясно,ясный
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.00004,0.00000,0.0,0.0,0.000000,0.0
1,0.0,0.0,0.0,0.0,0.0,0.000043,0.0,0.0,0.000000,0.000165,...,0.000084,0.0,0.0,0.000000,0.00000,0.00000,0.0,0.0,0.000037,0.0
2,0.0,0.0,0.0,0.0,0.0,0.000041,0.0,0.0,0.000060,0.000000,...,0.000000,0.0,0.0,0.000000,0.00000,0.00000,0.0,0.0,0.000000,0.0
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000056,0.000000,...,0.000000,0.0,0.0,0.000000,0.00000,0.00000,0.0,0.0,0.000033,0.0
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.000139,0.00000,0.00006,0.0,0.0,0.000038,0.0


2018-03-04 15:27:44,030| ERROR   | Could not establish connection from ('127.0.0.1', 39837) to remote side of the tunnel


In [58]:
lvl1 = pd.DataFrame(columns=list(lvl1_names))
defoult_nicht = {
    'lvl1_бизнес' : 0,
 'lvl1_культура' : 0,
 'lvl1_медиа' : 0,
 'lvl1_наука_и_техника' : 0,
 'lvl1_общество_и_путешествия' : 0,
 'lvl1_политика' : 0,
 'lvl1_происшествия' : 0,
 'lvl1_силовые_структуры' : 0,
 'lvl1_спорт' : 0,
 'lvl1_экономика_и_финансы' : 0,
 'total' : 0}
lvl1

,lvl1_наука_и_техника,lvl1_политика,lvl1_силовые_структуры,lvl1_экономика_и_финансы,lvl1_медиа,lvl1_спорт,lvl1_происшествия,lvl1_бизнес,lvl1_общество_и_путешествия,lvl1_культура


In [95]:
lvl1 = pd.DataFrame(columns=list(lvl1_names))
start_time = datetime.datetime(2017, 8, 1, 0, 0, 0)
delta = datetime.timedelta(hours=2)

while start_time + delta < datetime.datetime(2017, 9, 1, 0, 0, 0):
    cursor = db.documents.find({"rss_pubDate" : {'$gt' : start_time, '$lt' : start_time + delta}}).limit(500)
    cur_dic = {
    'lvl1_бизнес' : 0,
 'lvl1_культура' : 0,
 'lvl1_медиа' : 0,
 'lvl1_наука_и_техника' : 0,
 'lvl1_общество_и_путешествия' : 0,
 'lvl1_политика' : 0,
 'lvl1_происшествия' : 0,
 'lvl1_силовые_структуры' : 0,
 'lvl1_спорт' : 0,
 'lvl1_экономика_и_финансы' : 0,
 'total' : 0}
    for row in cursor:
        
        if len(row['topics']) <= 0:
            continue
            
        cur_dic['total'] += 1
        cur_dic[row['topics'][0]['lvl1']['id']] += row['topics'][0]['probability']
    lvl1 = lvl1.append(cur_dic, ignore_index=True)
    start_time += delta

In [96]:
buffer = lvl1

buffer = buffer.drop(columns='total', axis=0)

In [97]:
buffer = buffer.apply(lambda x : x / np.sum(x.values), axis=1)

In [98]:
buffer.to_csv('august_2h_500.csv')

In [227]:
lvl1.to_csv('./first_54_h.csv')

In [ ]:
buffer = pd.read_csv("./august_1h.csv", index_col='ID')

buffer.head()

In [140]:
import xgboost

In [ ]:
xgboost.XGBRegressor(max_depth=5)